***Cell 1: Setup - Imports e Configuração da API***

In [1]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output

# Configure API Key
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
if not google_api_key:
    raise ValueError("Google API key not found in .env file.")
genai.configure(api_key=google_api_key)
print("✅ Google Gemini client configured successfully.")

✅ Google Gemini client configured successfully.


***Cell 2: O Painel de Controlo (Widgets Interativos)***

In [2]:
# Model Selector with updated, valid model names
model_selector = widgets.Dropdown(
    options=['gemini-2.5-flash', 'gemini-1.5-flash-latest', 'gemini-2.5-pro'],
    value='gemini-1.5-flash-latest',
    description='Model:',
)

# Temperature Slider now goes up to 2.0
temperature_slider = widgets.FloatSlider(
    value=0.7, min=0, max=2.0, step=0.1, description='Temperature:', readout_format='.1f'
)

# Max Tokens Slider with a safe range
max_tokens_slider = widgets.IntSlider(
    value=2048, min=256, max=8192, step=256, description='Max Tokens:'
)

# Prompting Text Areas
system_prompt_input = widgets.Textarea(
    value='You are a helpful and creative AI assistant.',
    description='System Prompt:', layout={'height': '100px', 'width': '100%'}
)
user_prompt_input = widgets.Textarea(
    value="Explain the concept of 'prompt engineering' to a junior developer, providing a simple analogy.",
    description='User Prompt:', layout={'height': '200px', 'width': '100%'}
)

# Execution Button & Output Area
run_button = widgets.Button(description='Generate Response', button_style='success', icon='rocket')
output_area = widgets.Output()

# Display the control panel
print("--- Prompt Engineering Playground ---")
display(model_selector, temperature_slider, max_tokens_slider, system_prompt_input, user_prompt_input, run_button, output_area)

--- Prompt Engineering Playground ---


Dropdown(description='Model:', index=1, options=('gemini-2.5-flash', 'gemini-1.5-flash-latest', 'gemini-2.5-pr…

FloatSlider(value=0.7, description='Temperature:', max=2.0, readout_format='.1f')

IntSlider(value=2048, description='Max Tokens:', max=8192, min=256, step=256)

Textarea(value='You are a helpful and creative AI assistant.', description='System Prompt:', layout=Layout(hei…

Textarea(value="Explain the concept of 'prompt engineering' to a junior developer, providing a simple analogy.…

Button(button_style='success', description='Generate Response', icon='rocket', style=ButtonStyle())

Output()

***Cell 3: A Lógica de Execução (O "Motor")***

In [3]:
def generate_response_with_streaming(b):
    """Triggers on button click, calls Gemini API with streaming."""
    with output_area:
        clear_output()
        print("🚀 Sending request to Gemini...")

        # Gather values from widgets
        selected_model = model_selector.value
        generation_config = {
            "temperature": temperature_slider.value,
            "max_output_tokens": max_tokens_slider.value,
        }
        
        try:
            # Initialize the model with the system prompt
            model = genai.GenerativeModel(
                selected_model,
                system_instruction=system_prompt_input.value
            )

            # Generate content with stream=True
            response_stream = model.generate_content(user_prompt_input.value, stream=True)

            # --- STREAMING AND OUTPUT HANDLING ---
            clear_output()
            print("--- ✅ Gemini's Streaming Response ---")
            
            # Create an HTML widget to display the streaming text
            html_output = widgets.HTML(value="")
            display(html_output)
            
            full_response_text = ""
            # Iterate through the chunks of the stream
            for chunk in response_stream:
                if chunk.text:
                    full_response_text += chunk.text
                    # Update the HTML widget's value to create the streaming effect
                    html_output.value = full_response_text.replace('\n', '<br>')
            
            # Final check for empty response after stream is complete
            if not full_response_text.strip():
                html_output.value = "<i>O modelo não retornou nenhum conteúdo. Tente um prompt diferente ou verifique as configurações de segurança da sua API Key.</i>"

        except Exception as e:
            # Catch errors like invalid token limits or other API issues
            clear_output()
            print(f"--- ❌ An error occurred ---")
            display(widgets.HTML(value=f"<p style='color:red;'>{e}</p>"))

***Cell 4: Ligar o Botão à Lógica***

In [4]:
# This line "connects" the button to our function.
# When the button is clicked, the 'generate_response' function will be executed.
run_button.on_click(generate_response_with_streaming)